In [8]:
import os
import json
import pandas as pd
df = pd.read_csv('data.csv')

In [9]:
#df.set_index('Unnamed: 0', inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# df.index.name = None
df.head()

,title,publicationName,abstract,keywords,subjectArea,publication_date
0,Effects of iron content on the microstructure ...,Materials Chemistry and Physics,microstructure corrosion behavior hot rolled t...,EIS Microstructure Pitting corrosion Polarizat...,MATE,01/10/2018
1,The critical factors of research and innovatio...,International Journal of Trade and Global Markets,copyright inderscience enterprises ltd. univ...,Critical factors Innovation creation Public un...,BUSI,01/01/2018
2,Is the occiput-wall distance valid and reliabl...,Musculoskeletal Science and Practice,background hyperkyphosis frequently found nowa...,Cobb angle Dowager's hump Round back Spine,HEAL,01/12/2018
3,Comparison of soil composition between farmlan...,Eurasian Journal of Analytical Chemistry,society innovative research rights reserved ...,Agriculture land management Conserved area Soi...,CHEM,01/01/2018
4,The influence of neighbor effect and urbanizat...,Progress in Transplantation,natco rights reserved introduction population ...,Choice Consumer wellness Decision-making Neigh...,MEDI,01/03/2018


In [3]:
df.shape

(16319, 7)

In [11]:
df.isna().sum()

title               0
publicationName     0
abstract            0
keywords            0
subjectArea         0
publication_date    0
dtype: int64

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import compute_class_weight
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# Assuming you have a DataFrame 'df' with columns: title, publicationName, abstract, keywords, subjectArea

# Text preprocessing and vectorization
text_data = df['title'] + ' ' + df['publicationName'] # + ' ' + df['keywords']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)
y = df['subjectArea']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_encoded

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [15]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

{0: np.float64(0.5525925925925926),
 1: np.float64(2.878086419753086),
 2: np.float64(0.5354579385587137),
 3: np.float64(1.6847335140018067),
 4: np.float64(0.9054653904841171),
 5: np.float64(0.6337071015970098),
 6: np.float64(0.5839595634281625),
 7: np.float64(30.21990740740741),
 8: np.float64(2.313485734538366),
 9: np.float64(2.429741299088033),
 10: np.float64(4.435949711179069),
 11: np.float64(1.176444083986663),
 12: np.float64(0.7079334092511252),
 13: np.float64(0.8759393451422437),
 14: np.float64(4.278924942641757),
 15: np.float64(1.2210063598952487),
 16: np.float64(0.7519728126260008),
 17: np.float64(2.4669312169312168),
 18: np.float64(0.17537849783043835),
 19: np.float64(1.9735449735449735),
 20: np.float64(2.3701888162672478),
 21: np.float64(4.518864659051575),
 22: np.float64(1.3139090177133654),
 23: np.float64(0.98476276684016),
 24: np.float64(4.31712962962963),
 25: np.float64(0.7965708707059613),
 26: np.float64(1.7330412850126111)}

In [16]:
# Create and train XGBoost model
model = XGBClassifier(scale_pos_weight=class_weight_dict) # scale_pos_weight=class_weight_dict
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [00:02:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.82      0.75      0.79       198
           1       0.91      0.78      0.84        40
           2       0.72      0.60      0.65       207
           3       0.70      0.58      0.63        64
           4       0.88      0.80      0.84       159
           5       0.79      0.79      0.79       200
           6       0.79      0.85      0.82       210
           7       1.00      0.22      0.36         9
           8       0.82      0.69      0.75        48
           9       0.79      0.61      0.69        38
          10       0.62      0.36      0.45        28
          11       0.78      0.81      0.80        96
          12       0.79      0.72      0.75       208
          13       0.77      0.79      0.78       131
          14       0.80      0.29      0.42        28
          15       0.86      0.73      0.79       113
          16       0.81      0.84      0.82       170
          17       0.87    